## Create batch pipeline

In [1]:
# Connect to your workspace
from azureml.core import Workspace
ws = Workspace.from_config()

## Provision inference compute

In [2]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

cluster_name = "mlopsbootcamp"

try:
    # Check for existing compute target
    inference_cluster = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    # If it doesn't already exist, create it
    try:
        compute_config = AmlCompute.provisioning_configuration(vm_size='Standard_DS2_v2', max_nodes=2)
        inference_cluster = ComputeTarget.create(ws, cluster_name, compute_config)
        inference_cluster.wait_for_completion(show_output=True)
    except Exception as ex:
        print(ex)

Found existing cluster, use it.


## Create a pipeline for batch inferencing

In [3]:
import os
# Create a folder for the experiment files
experiment_folder = 'batch_pipeline'
os.makedirs(experiment_folder, exist_ok=True)

print(experiment_folder)

batch_pipeline


## Get path to the model

In [4]:
from azureml.core import Model
model_list = Model.list(ws)
model_path = Model.get_model_path('fourier_regression')
print(model_list, model_path)

[Model(workspace=Workspace.create(name='mlworkspace', subscription_id='e7d71274-b7c4-47ed-9751-2505b563b918', resource_group='mlgroup'), name=fourier_regression, id=fourier_regression:1, version=1, tags={}, properties={})] azureml-models\fourier_regression\2\fourier.pkl


## Load the model

In [5]:
import joblib
model = joblib.load(model_path)
model

LinearRegression()

## Check the batch data

In [6]:
mini_batch = list()
for (dirpath, dirnames, filenames) in os.walk("batch-data"):
    mini_batch += [os.path.join(dirpath, file) for file in filenames]
for elem in mini_batch:
    print(elem)

batch-data\1.csv
batch-data\2.csv
batch-data\3.csv
batch-data\4.csv
batch-data\5.csv
batch-data\6.csv
batch-data\7.csv


## Make prediction per batch

In [7]:
import numpy as np
def run(mini_batch):
    # This runs for each batch
    resultList = []

    # process each file in the batch
    for f in mini_batch:
        # Read comma-delimited data into an array
        data = np.genfromtxt(f, delimiter=',')
        print(data)
        # Reshape into a 2-dimensional array for model input
        prediction = model.predict(data.reshape(1, -1))
        # Append prediction to results
        resultList.append("{}: {}".format(os.path.basename(f), prediction[0]))
    return resultList

result = run(mini_batch)
result

[2.88624084e+02 0.00000000e+00 2.58819045e-01 9.65925826e-01
 5.00000000e-01 8.66025404e-01 7.07106781e-01 7.07106781e-01
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
[2.88446014e+02 0.00000000e+00 2.58819045e-01 9.65925826e-01
 5.00000000e-01 8.66025404e-01 7.07106781e-01 7.07106781e-01
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
[2.88254852e+02 0.00000000e+00 2.58819045e-01 9.65925826e-01
 5.00000000e-01 8.66025404e-01 7.07106781e-01 7.07106781e-01
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
[2.88050537e+02 0.00000000e+00 2.58819045e-01 9.65925826e-01
 5.00000000e-01 8.66025404e-01 7.07106781e-01 7.07106781e-01
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
[2.87846252e+02 0.00

['1.csv: 95.15347618619145',
 '2.csv: 95.2173844857991',
 '3.csv: 95.28599144188169',
 '4.csv: 95.35931895959774',
 '5.csv: 95.43263552473465',
 '6.csv: 95.5059630424507',
 '7.csv: 95.40248307420845']

## Summarize into python script

In [8]:
%%writefile $experiment_folder\score.py

import os
import numpy as np
from azureml.core import Model
import joblib

def init():
    # Runs when the pipeline step is initialized
    global model

    # load the model
    model_path = Model.get_model_path('fourier_regression')
    model = joblib.load(model_path)

def run(mini_batch):
    # This runs for each batch
    resultList = []

    # process each file in the batch
    for f in mini_batch:
        # Read comma-delimited data into an array
        data = np.genfromtxt(f, delimiter=',')
        # Reshape into a 2-dimensional array for model input
        prediction = model.predict(data.reshape(1, -1))
        # Append prediction to results
        resultList.append("{}: {}".format(os.path.basename(f), prediction[0]))
    return resultList

Overwriting batch_pipeline\score.py


## Create conda environment for the pipeline

In [9]:
%%writefile $experiment_folder\batch_environment.yml
name: batch_environment
dependencies:
- python=3.8
- numpy
- pandas
- scikit-learn
- pip:
    - azureml-core
    - azureml-dataset-runtime[fuse]
    - azureml-pipeline-core
    - azureml-pipeline-steps

Overwriting batch_pipeline\batch_environment.yml


## Define run using environment

In [10]:
from azureml.core import Environment
from azureml.core.runconfig import DEFAULT_CPU_IMAGE

# Create an Environment for the experiment
batch_env = Environment.from_conda_specification("experiment_env", experiment_folder + "/batch_environment.yml")
batch_env.docker.base_image = DEFAULT_CPU_IMAGE
print('Configuration ready.')

Configuration ready.


## Configure batch pipeline steps

In [13]:
from datetime import datetime

from azureml.pipeline.steps import ParallelRunConfig, ParallelRunStep
from azureml.data import OutputFileDatasetConfig
from azureml.core.runconfig import DockerConfiguration

# Get the batch dataset for input
batch_data_set = ws.datasets['batch-data']

# Set the output location
default_ds = ws.get_default_datastore()
output_dir = OutputFileDatasetConfig(name='inferences')

# Define the parallel run step step configuration
parallel_run_config = ParallelRunConfig(
    source_directory=experiment_folder,
    entry_script="score.py",
    mini_batch_size="5",
    error_threshold=10,
    output_action="append_row",
    environment=batch_env,
    compute_target=inference_cluster,
    node_count=2)

parallel_step_name = "batchscoring-" + datetime.now().strftime("%Y%m%d%H%M")

# Create the parallel run step
parallelrun_step = ParallelRunStep(
    name=parallel_step_name,
    parallel_run_config=parallel_run_config,
    inputs=[batch_data_set.as_named_input('batch_data')],
    output=output_dir,
    arguments=[],
    allow_reuse=True
)

print('Steps defined')

Steps defined


## Run the pipeline

In [14]:
from azureml.core import Experiment
from azureml.pipeline.core import Pipeline

# Create the pipeline
pipeline = Pipeline(workspace=ws, steps=[parallelrun_step])

# Run the pipeline as an experiment
pipeline_run = Experiment(ws, 'pytown-energy-demand-batch').submit(pipeline)
pipeline_run.wait_for_completion(show_output=True)

Created step batchscoring-202108011821 [cebb0cad][0d816a9e-0dfb-4875-bf71-f04ba0986734], (This step will run and generate new outputs)
Submitted PipelineRun fe15a4d6-dabf-4a1b-a90a-a6294bf139ef
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/fe15a4d6-dabf-4a1b-a90a-a6294bf139ef?wsid=/subscriptions/e7d71274-b7c4-47ed-9751-2505b563b918/resourcegroups/mlgroup/workspaces/mlworkspace&tid=a0f1cacd-618c-4403-b945-76fb3d6874e5
PipelineRunId: fe15a4d6-dabf-4a1b-a90a-a6294bf139ef
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/fe15a4d6-dabf-4a1b-a90a-a6294bf139ef?wsid=/subscriptions/e7d71274-b7c4-47ed-9751-2505b563b918/resourcegroups/mlgroup/workspaces/mlworkspace&tid=a0f1cacd-618c-4403-b945-76fb3d6874e5
PipelineRun Status: NotStarted
PipelineRun Status: Running


StepRunId: 26c56c9e-7c28-4fac-8502-605f31b01679
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/26c56c9e-7c28-4fac-8502-605f31b01679?wsid=/subscriptions/e7d71274-b7c4-47ed-9751-2505b56

'Finished'

## Retrieve predictions

In [15]:
import pandas as pd
import shutil

# Remove the local results folder if left over from a previous run
shutil.rmtree('batch-results', ignore_errors=True)

# Get the run for the first step and download its output
prediction_run = next(pipeline_run.get_children())
prediction_output = prediction_run.get_output_data('inferences')
prediction_output.download(local_path='batch-results')

# Traverse the folder hierarchy and find the results file
for root, dirs, files in os.walk('batch-results'):
    for file in files:
        if file.endswith('parallel_run_step.txt'):
            result_file = os.path.join(root,file)

# cleanup output format
df = pd.read_csv(result_file, delimiter=":", header=None)
df.columns = ["File", "Prediction"]

# Display the first 20 results
df.head(20)

,File,Prediction
0,1.csv,95.153476
1,2.csv,95.217384
2,3.csv,95.285991
3,4.csv,95.359319
4,5.csv,95.432636
5,6.csv,95.505963
6,7.csv,95.402483


## Load predictions into data storage

In [24]:
datastore = ws.get_default_datastore()
datastore.upload(src_dir="batch-results", target_path="batch-results", overwrite=True, show_progress=True)


Uploading an estimated of 1 files
Uploading batch-results\dataset\26c56c9e-7c28-4fac-8502-605f31b01679\inferences\parallel_run_step.txt
Uploaded batch-results\dataset\26c56c9e-7c28-4fac-8502-605f31b01679\inferences\parallel_run_step.txt, 1 files out of an estimated total of 1
Uploaded 1 files


$AZUREML_DATAREFERENCE_2fa9834b956a4bd3abf94d07eb06a4a4

## Publish the pipeline

In [16]:
published_pipeline = pipeline_run.publish_pipeline(name='Fourier_regression_batch_prediction_pipeline',
                                                   description='Batch scoring using linear regression model with Fourier ML features',
                                                   version='1.0')

published_pipeline

Name,Id,Status,Endpoint
Fourier_regression_batch_prediction_pipeline,2304f3ec-8493-4aed-b57c-e421876b0c81,Active,REST Endpoint


## Get REST endpoint

In [17]:
rest_endpoint = published_pipeline.endpoint
print(rest_endpoint)

https://westeurope.api.azureml.ms/pipelines/v1.0/subscriptions/e7d71274-b7c4-47ed-9751-2505b563b918/resourceGroups/mlgroup/providers/Microsoft.MachineLearningServices/workspaces/mlworkspace/PipelineRuns/PipelineSubmit/2304f3ec-8493-4aed-b57c-e421876b0c81


## Schedule the pipeline to run every Monday at 04:00 in the morning (02:00 UTC)

In [18]:
from azureml.pipeline.core import ScheduleRecurrence, Schedule

weekly = ScheduleRecurrence(frequency='Week', interval=1, week_days=["Monday"], time_of_day="02:00")
pipeline_schedule = Schedule.create(ws, name='Weekly Predictions',
                                        description='batch inferencing',
                                        pipeline_id=published_pipeline.id,
                                        experiment_name='Batch_Prediction',
                                        recurrence=weekly)

## Disable pipeline with active schedule

In [22]:
ss = Schedule.list(ws)
for s in ss:
    print(s)

Pipeline(Name: Weekly Predictions,
Id: d27dcbf0-4c13-418c-95f6-a39a03882411,
Status: Active,
Pipeline Id: 2304f3ec-8493-4aed-b57c-e421876b0c81,
Pipeline Endpoint Id: None,
Recurrence Details: Runs at 2:00 on Monday every Week)


In [23]:
def stop_by_schedule_id(ws, schedule_id):
    s = next(s for s in Schedule.list(ws) if s.id == schedule_id)
    s.disable()
    return s

stop_by_schedule_id(ws, s.id)

Name,Id,Status,Pipeline Id,Pipeline Endpoint Id,Recurrence Details
Weekly Predictions,d27dcbf0-4c13-418c-95f6-a39a03882411,Disabled,2304f3ec-8493-4aed-b57c-e421876b0c81,,Runs at 2:00 on Monday every Week
